<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Real_state_url_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.google.com/search?q=املاک+تهران+اجاره"
r = requests.get(url)
page = r.text 
soup = BeautifulSoup(page, 'lxml') 

i = 0

link_list = []
for tag in soup.find_all('a'):
    i+=1
    href = tag['href']
    if re.search('http',href):
        try:
            link = re.search('https://.+\.com',href).group(0)
            link_list.append(link)
        except:
            pass

link_list = list(set(link_list))

link_list2 = [] 

for link in link_list:
    if not re.search('google.com',link):
        link_list2.append(link)
        
print(link_list2)

import pandas as pd

df = pd.DataFrame(link_list2)
df.to_csv('/content/sample_data/bs4_final.csv', index=False)

['https://kilid.com', 'https://www.2nabsh.com', 'https://shabesh.com', 'https://www.zoomila.com', 'https://www.sheypoor.com']


In [10]:
df = pd.read_csv("/content/sample_data/bs4_final.csv" , delim_whitespace=True)#"/kaggle/input/japanese-property-urls/tokyo_property_urls.csv")
print(df)

                          0
0         https://kilid.com
1    https://www.2nabsh.com
2       https://shabesh.com
3   https://www.zoomila.com
4  https://www.sheypoor.com
